In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/titanicdataset-traincsv/train.csv')
df.head()
df.describe()
df.info()
df['Embarked']
avg_age = df["Age"].astype("float").mean(axis=0)
df["Age"].replace(np.nan, avg_age, inplace = True)
df["Age"] = df["Age"].astype("float")


In [ ]:
cat_feats = ['Sex','Embarked']
final_data = pd.get_dummies(df,columns=cat_feats,drop_first=True)
from sklearn.preprocessing import StandardScaler
standard_df = StandardScaler().fit_transform(final_data.drop(['Name','Ticket','Cabin'],axis =1))
standard_df.shape
from sklearn import decomposition
pca=decomposition.PCA()
pca.n_components=9
pca_df=pca.fit_transform(standard_df)
df=pd.DataFrame(pca_df)
df

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4,random_state = 15)
kmeans.fit(df)


In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'n_clusters': [2,4,6,8]} 
grid = GridSearchCV(KMeans(),param_grid,refit=True,verbose=2)
grid.fit(df)

grid.best_params_

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report
kmeans = KMeans(n_clusters=8,n_init = 10, init = 'random')
kmeans.fit(df)

erorr = []
for i in range(1,11):
    kmeans = KMeans(n_clusters=i,n_init = 10, init = 'k-means++',max_iter = 300,random_state = 0)
    kmeans.fit(df)
    erorr.append(kmeans.inertia_)
plt.plot(range(1,11),erorr,marker = 'o')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=8,n_init = 10, init = 'k-means++',max_iter = 300,random_state = 0)
pred=kmeans.fit_predict(df)

labels=kmeans.labels_
print(labels)
print(max(labels))
print(min(labels))

In [ ]:
df['Cluster']=pd.Series(labels)
cluster_names={0:'Claster 0',
              1:'Claster 1',
              2:'Claster 2',
              3:'Claster 3',
              4:'Claster 4',
              5:'Claster 5',
              6:'Claster 6',
              7:'Claster 7',
              }
df['Cluster_names']=df['Cluster'].map(cluster_names)

plt.figure(figsize=(10,6))
df[df['Cluster']== 0]['Cluster_names'].hist(alpha=0.5,
                                              bins=30,label='0')
df[df['Cluster']==1]['Cluster_names'].hist(alpha=0.5,
                                              bins=30,label='1')
df[df['Cluster']== 2]['Cluster_names'].hist(alpha=0.5,
                                              bins=30,label='2')
df[df['Cluster']==3]['Cluster_names'].hist(alpha=0.5,
                                              bins=30,label='3')
df[df['Cluster']== 4]['Cluster_names'].hist(alpha=0.5,
                                              bins=30,label='4')
df[df['Cluster']==5]['Cluster_names'].hist(alpha=0.5,
                                              bins=30,label='5')
df[df['Cluster']== 6]['Cluster_names'].hist(alpha=0.5,
                                              bins=30,label='6')
df[df['Cluster']==7]['Cluster_names'].hist(alpha=0.5,
                                              bins=30,label='7')
plt.legend()

In [ ]:
import plotly.express as px
%matplotlib inline 
scatter=px.scatter_3d(x=0,y=1,z=2,data_frame=df[:],color=df['Cluster_names'])
scatter.show()